In [1]:
# !pip install -U gensim
# # !pip install nltk
# # !pip install spacy
# !install --upgrade scipy
# !pip install gensim[all]
# !python -m spacy download 'en_core_web_sm'
from gensim.test.utils import datapath

from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import spacy
import numpy as np
import tqdm 
import pandas as pd
import tqdm
import pandas as pd

nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# # # !pip uninstall scipy -y
# # !pip install scipy
# # # !pip install --upgrade pip
# !pip install scipy==1.10.0

# !pip install gensim 

In [ ]:
STOPWORDS

In [3]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[:3]


In [4]:
len(data_tip)

3

In [5]:
tip_text = data_tip['text']

In [6]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [7]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



100%|██████████| 9/9 [00:00<00:00, 94371.84it/s]


In [8]:
tip_text_preprocessed

0                     [avenger, time, lady]
1    [good, desert, tasty, cuban, sandwich]
2                             [open, think]
Name: text, dtype: object

In [9]:
model3= FastText(vector_size=300,workers=20,window=15,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=10000)

In [10]:
model3.save('/kaggle/working/custom_trained_model.h5')

In [11]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [12]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience".split()
test_words=list(np.array(test_words).astype(str))

In [13]:
def test_words_andmodel_updating(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
        model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [15]:
test_custom_trained_model_updating=test_words_andmodel_updating(model3,test_words)

In [16]:
test_custom_trained_model_updating

,test_embedding,most_similar,most_opposite
0,"[0.00052292267, 0.0002602252, 0.0002512252, -0...","[(open, 0.11268849670886993), (think, 0.108660...","[(good, -0.10464377701282501), (sandwich, -0.1..."
1,"[-0.004709644, 0.009207326, -0.0013622496, -0....","[(avenger, 0.9974217414855957), (lady, 0.99710...","[(sandwich, -0.9952232241630554), (good, -0.99..."
2,"[-0.00016820026, 0.0011945283, -9.233079e-05, ...","[(lady, 0.00031149317510426044), (good, -0.000...","[(time, 0.0018848528852686286), (open, 0.00180..."
3,"[0.00011421845, 8.909103e-05, -0.00038052493, ...","[(sandwich, 0.041897837072610855), (cuban, 0.0...","[(time, -0.03387967124581337), (avenger, -0.03..."
4,"[-0.0001561204, 8.371378e-05, 0.0003431353, -2...","[(tasty, 0.02631758339703083), (cuban, 0.02631...","[(lady, -0.020065592601895332), (think, -0.021..."
5,"[-3.881714e-06, -6.073078e-05, -0.00011706338,...","[(good, -0.038615211844444275), (time, -0.0387...","[(open, 0.04342029616236687), (cuban, 0.041856..."
6,"[-0.00061720965, 0.00015287117, 0.0006066284, ...","[(lady, 0.016121847555041313), (avenger, 0.015...","[(think, -0.010440894402563572), (cuban, -0.01..."
7,"[0.000816348, -0.0003564333, -0.00036577682, 6...","[(sandwich, -0.02027047984302044), (good, -0.0...","[(lady, 0.027015557512640953), (time, 0.026492..."
8,"[-0.00071926456, 1.5520834e-05, -0.00015091502...","[(time, -0.034877702593803406), (think, -0.035...","[(tasty, 0.036862339824438095), (open, 0.03678..."
9,"[-0.00027109464, 0.00012610167, 0.00026256018,...","[(lady, 0.005031847395002842), (avenger, 0.004...","[(open, 0.0006511926185339689), (good, -0.0020..."


In [17]:

pd.set_option('display.max_colwidth', None)
test_custom_trained_model_updating.to_csv('/kaggle/working/my_trained_model_test__.csv', index=False)


In [18]:
test_custom_trained_model_updating

,test_embedding,most_similar,most_opposite
0,"[0.00052292267, 0.0002602252, 0.0002512252, -0.00026776252, -0.00023149766, -0.0004391262, 0.0006524005, -0.00050169503, 0.0017225484, -0.0003192834, 0.00027631593, 0.0012040482, 0.00029999652, 0.00045472322, -0.0002919829, -0.00091873255, 0.0008217223, 0.0001683199, -0.00058104267, -0.00015962143, 0.00034758108, 0.00033434122, 0.00025976155, -0.00090911175, -0.00025277399, 0.0001054988, -0.00043315094, 0.0005087248, 0.00046520028, 0.00053886435, -0.00018257594, 0.00017940695, -0.00013449573, 6.285288e-05, 0.0010167569, 0.00028696549, 0.00017555771, -0.0005886973, 0.0001953924, 0.00051383727, 0.0001086723, 0.0014225495, 0.00032838833, -0.00084408664, -7.298955e-05, 0.00065406365, 0.00020955972, 0.0006340929, 0.0005995781, 0.0005003697, 0.0006645769, -0.00017995165, -3.6864025e-05, 0.00028201877, -0.0007647447, -0.000503735, 0.00076796446, 0.00046891245, 0.00033174237, 0.0007600947, -0.00022129873, -0.00069202157, -0.00041655186, -3.3602118e-06, 3.2044275e-06, 0.0007835606, -0.00058605836, 0.0008047783, 0.00053757115, -0.00039472897, 0.0010019037, -0.00073443697, 0.0016163074, -0.00025672108, 0.00029498144, -0.00024799805, 0.0009810645, 0.00022384113, 0.0006417596, 0.00014812931, 9.757401e-05, 0.00078320486, 0.0001942244, 2.6167354e-05, -0.00021760106, -0.0006115989, -0.00023360821, 0.0003408869, -0.00073422276, 0.00027625376, -0.00019004721, 0.001309748, -0.0009328896, 0.000537742, -0.00087608706, 0.00046694968, 0.00080599956, -0.0002867826, 8.875835e-05, 0.00022704616, ...]","[(open, 0.11268849670886993), (think, 0.10866012424230576), (avenger, 0.10836189985275269), (lady, 0.10663307458162308), (tasty, 0.1056782603263855), (cuban, 0.10529149323701859), (time, 0.10505804419517517), (desert, 0.10503323376178741), (sandwich, 0.10494256019592285), (good, 0.10464377701282501)]","[(good, -0.10464377701282501), (sandwich, -0.10494256019592285), (desert, -0.10503323376178741), (time, -0.10505804419517517), (cuban, -0.10529149323701859), (tasty, -0.1056782603263855), (lady, -0.10663307458162308), (avenger, -0.10836189985275269), (think, -0.10866012424230576), (open, -0.11268849670886993)]"
1,"[-0.004709644, 0.009207326, -0.0013622496, -0.0032375217, -0.016568959, -0.002217721, -0.005891438, 0.0049285213, 0.0036447337, -0.00030818643, -0.0012302483, 0.0043985946, -0.004803267, 0.011184551, -0.0056073796, 0.00065049477, -0.00071702484, -0.0004792783, 0.0016978416, 0.020405153, -0.003336369, -0.0045959163, 0.0058406983, -0.012405338, 0.0027388865, 0.008943662, -0.0058606607, 0.0024720142, 0.008348522, -0.009145501, 0.0054993723, 0.004426308, -0.00051200256, -0.0055150576, -3.9711485e-05, -0.0076146517, 0.009152727, -0.0017863975, 0.0072324434, -0.0032232644, 0.007456722, -0.0008231347, -0.003644327, -0.01260085, -0.00037006338, -0.0027757788, -0.015166019, 0.010110216, 0.0031097366, 0.0034908678, 0.007000987, 0.008259876, -0.001972469, 0.00872817, -0.002193481, -0.004209239, 0.004605583, -0.0021442159, 0.0025938696, -0.00028987057, -0.0053931912, -0.0012591123, -0.002796219, 0.004881996, 0.00795398, -0.00334261, 0.0017196873, 0.0037727612, -0.010256405, -0.0075284755, 0.0072730547, 0.00062756083, 0.0006100489, -0.0016491933, 0.0060746837, -0.00039244085, -0.0015171715, 0.0054860134, -0.0062205577, -0.00033736348, -0.00067794195, 0.001772813, -0.00087350816, 0.010273547, 0.011628651, 0.0057141655, 0.0076369867, -0.012685462, -0.004242699, -0.001587301, 0.006698012, -0.00880938, -0.008664487, -0.005736349, -0.00048083792, -0.0065043345, 0.004984665, -0.004885792, 0.003979239, 0.0025230965, ...]","[(avenger, 0.9974217414855957), (lady, 0.9971053004264832), (time, 0.9967055916786194), (think, 0.996147871017456), (open, 0.9958803653717041), (tasty, 0.9956443309783936), (cuban, 0.9954566359519958), (desert, 0.9954541325569153), (good, 0.9953598976135254), (sandwich, 0.9952232241630554)]","[(sandwich, -0.9952232241630554), (good, -0.9953598976135254), (desert, -0.9954541325569153), (cuban, -0.995456

In [19]:
np.array(test_custom_trained_model_updating['test_embedding'][0]).shape

(300,)

# Facebook pretrained model

In [22]:
!pip install fasttext[all]

  Using cached fasttext-0.9.2.tar.gz (68 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4563132 sha256=a69c22f40eb8ca625d3522390015431ed343b3688e9a110a9fc6fc3814adfbfa
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [23]:
import fasttext
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore') 
model_path='/kaggle/working/cc.en.300.bin.gz'


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import gensim
import gzip
with gzip.open('/kaggle/working/cc.en.300.bin.gz', 'rb') as f:
      f.read()
 


In [32]:
from gensim.models import fasttext

model_pre_trained = fasttext.load_facebook_model('/kaggle/working/cc.en.300.bin')

In [36]:
model_pre_trained.wv['hello']

array([ 1.57576188e-01,  4.37820926e-02, -4.51271934e-03,  6.65931404e-02,
        7.70346820e-02,  4.85855248e-03,  8.19822028e-03,  6.52402919e-03,
        9.25899856e-03,  3.53899002e-02, -2.31395271e-02, -4.91807126e-02,
       -8.32642540e-02,  1.56014524e-02,  2.54856616e-01,  3.45423706e-02,
       -1.07451361e-02, -7.80188590e-02, -7.08099529e-02,  7.62385577e-02,
       -6.09613657e-02,  4.48625796e-02, -7.29744136e-02,  1.30583309e-02,
        3.14881057e-02, -3.10055036e-02,  1.66004002e-02,  1.74405202e-02,
       -7.35838860e-02,  1.18252613e-01, -1.21330231e-01, -4.09253240e-02,
        2.93969568e-02,  4.84445989e-02, -1.33816330e-02, -1.74765270e-02,
        7.51308873e-02,  9.97046307e-02, -4.00476977e-02,  4.05735290e-03,
       -7.21896589e-02, -4.43356819e-02, -1.22628408e-03,  7.56693557e-02,
        3.98401320e-02,  3.22643593e-02,  1.95914153e-02,  4.68016043e-02,
       -1.46228177e-02,  1.12967767e-01,  3.15065160e-02, -1.02312110e-01,
        1.58124104e-01, -

In [39]:
def test_words_pretrain_model(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
#         model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_word":test_words,"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [41]:
pd.set_option('display.max_colwidth', None)
pretrain_model_testing=test_words_pretrain_model(model_pre_trained,test_words)
pretrain_model_testing

,test_word,test_embedding,most_similar,most_opposite
0,Pizza,"[-0.04730315, 0.055486925, 0.002312106, 0.008184079, 0.010215392, 0.10119636, 0.0036317892, -0.01563866, 0.06815292, 0.05020129, 0.049392235, -0.043568533, -0.09103884, 0.06379739, -0.0013348646, 0.13690172, -0.047353666, -0.07072039, -0.031746037, 0.02378206, -0.035532415, 0.057132464, -0.09186747, 0.0021070922, 0.0543589, 0.045041777, 0.16277888, -0.17310376, -0.15073496, 0.15644233, -0.010509309, 0.2475494, -0.04142794, 0.050304323, -0.12183672, -0.03074091, -0.078320354, 0.0400592, -0.028893083, -0.18614209, -0.062400445, 0.052418318, -0.06635752, 0.08446585, -0.0855748, -0.015535153, 0.031017685, -0.06620627, 0.09153956, 0.15365931, -0.0044119097, 0.008067329, -0.007055682, 0.120593935, -0.11484016, -0.017348692, 0.003464643, -0.10059928, 0.023931228, 0.0064523667, 0.027424246, -0.13463007, -0.11315498, 0.23674907, -0.029942598, -0.010420881, 0.033425998, 0.048952747, 0.029021606, -0.028985927, 0.20072824, 0.0292092, 0.07583181, 0.0057301708, -0.011138481, 0.065777846, -0.11353917, 0.11080846, -0.053976588, -0.102834545, 0.0057630716, 0.002462836, -0.05281395, -0.016304508, -0.021875558, 0.025778048, 0.04002054, 0.059978947, 0.04449929, 0.09152286, 0.011056645, 0.016909506, -0.0062631257, 0.07885225, 0.032587986, 0.09577962, -0.05134695, 0.12389119, 0.040980067, 0.08553994, ...]","[(Pizzeria, 0.7960581183433533), (Pizzas, 0.7806003093719482), (Pizza-, 0.7779601812362671), (pizza, 0.7579522132873535), (Pizzaria, 0.7133852243423462), (Vocelli, 0.7081500291824341), (Schmizza, 0.6937696933746338), (Pizza1, 0.6842173337936401), (PIzza, 0.6744317412376404), (PizzaI, 0.6665058732032776)]","[(obviating, 0.1975397914648056), (DERIVATIVES, 0.1904284656047821), (Lúthien, 0.18722538650035858), (issus, 0.18641728162765503), (outbalanced, 0.1841462403535843), (attrited, 0.1806671917438507), (bleedover, 0.18056851625442505), (obviate, 0.17968051135540009), (backwashes, 0.179137721657753), (crosscurrent, 0.17609883844852448)]"
1,Burger,"[-0.05884499, -0.012373974, 0.026495267, 0.025184467, 0.03211139, 0.15347193, -0.014935261, -0.08232591, -0.061652202, -0.095548496, -0.017619409, -0.002346178, -0.07241221, 0.14038219, -0.086021736, 0.08426088, -0.054789275, -0.11063664, 0.014295248, 0.036194693, -0.03873261, -0.05876424, 0.014933603, 0.0073507456, 0.035449952, 0.047467124, 0.14763671, -0.049066067, -0.0033984322, 0.11014191, -0.045270335, 0.14974724, -0.065994814, -0.021238744, -0.07344714, -0.04618569, -0.027204433, 0.014503771, -0.13275222, -0.0847947, 0.03710636, 0.041173935, -0.11297399, 0.013547611, -0.008104327, -0.033364937, -0.07259293, -0.044350225, 0.04774462, 0.010031728, 0.0029619145, 0.034111746, -0.08580373, 0.053468596, -0.004321528, -0.14170542, -0.016526053, -0.04649473, 0.03351213, -0.0023403023, 0.0797544, -0.06629186, -0.031049263, 0.19140284, -0.010797964, 0.084136754, -0.054293483, -0.06260501, 0.0138751445, 0.060272865, 0.09655449, 0.0069928938, 0.022854155, -0.015271993, -0.0043674386, 0.006514509, -0.083905205, 0.056519486, -0.028758239, -0.042270668, -0.0499378, -0.07726429, 0.056359805, 0.004778209, 0.0046717436, 0.03429108, 0.010579422, 0.106799185, 0.024331931, 0.08221191, 0.08579481, 0.011966114, -0.033739954, 0.021161888, 0.00849455, 0.08558844, -0.045328476, 0.04727473, 0.059338044, -0.04453931, ...]","[(Burger-, 0.707100510597229), (burger, 0.7021234631538391), (Burgers, 0.6837388277053833), (Whopper, 0.652519166469574), (McBurger, 0.6466577649116516), (Cheeseburger, 0.638884961605072), (Fries, 0.6313252449035645), (McDonalds, 0.6107749342918396), (Buger, 0.609431803226471), (McDonald, 0.6078466176986694)]","[(activity., 0.20113711059093475), (soler, 0.18268610537052155), (refelect, 0.18006004393100739), (aura., 0.1779114454984665), (periods., 0.17727713286876678), (caed, 0.1763744205236435), (susceptibles, 0.17484529316425323), (characteristi, 0.17457729578018188), (characteristics., 0.17444857954978943), (valleys., 0.1741415113210678)]"
2,Co

In [42]:
pretrain_model_testing.to_csv('pretrain_model_testing')